In [1]:
import pandas as pd
from datetime import datetime as dt

In [14]:
#Path for reading the csv as a dataframe
df = pd.read_csv('Resources/hawaii_measurements.csv')
df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,NaN,73


## Bonus Challenge Assignment: Temperature Analysis I

In [15]:
# Convert the date column format from string to datetime

df['date'] = df[['date']].apply(pd.to_datetime)
df.head()


,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,NaN,73


In [45]:
# Set the date column as the DataFrame index
reset_df = df.set_index('date')

In [46]:
# Drop the date column - what was the point of this??
drop_df = reset_df.reset_index(drop = True)
drop_df

,station,prcp,tobs
0,USC00519397,0.08,65
1,USC00519397,0.00,63
2,USC00519397,0.00,74
3,USC00519397,0.00,76
4,USC00519397,NaN,73
...,...,...,...
19545,USC00516128,0.09,71
19546,USC00516128,NaN,78
19547,USC00516128,0.56,76
19548,USC00516128,0.50,76


### Compare June and December data across all years

In [47]:
from scipy import stats


In [48]:
# Filter data for desired months
#Filter for June (date column is the index)
june_df = reset_df[reset_df.index.month == 6]

#Filter for Dec(date column is the index)
dec_df = reset_df[reset_df.index.month == 12]

In [49]:
# Identify the average temperature for June
june_df['tobs'].mean()


74.94411764705882

In [50]:
# Identify the average temperature for December
dec_df['tobs'].mean()

71.04152933421226

In [73]:
#In order to do a paired t, I need equal numbers of temperature readings for June/Dec. Examine dataframes for entry number

june_df.info()
dec_df.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1700 entries, 2010-06-01 to 2017-06-30
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   station  1700 non-null   object 
 1   prcp     1574 non-null   float64
 2   tobs     1700 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 53.1+ KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1517 entries, 2010-12-01 to 2016-12-31
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   station  1517 non-null   object 
 1   prcp     1405 non-null   float64
 2   tobs     1517 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 47.4+ KB


In [74]:
#Need equal arrays to run a paired t-test. Obtain a random sampling of 1517 entries from June to match December.
from random import sample
june_sample_df = june_df.sample(1517)
june_sample_df, dec_df

(                station  prcp  tobs
 date                               
 2016-06-02  USC00513117  0.15    69
 2010-06-09  USC00514830  0.00    73
 2015-06-09  USC00519281  0.34    69
 2013-06-09  USC00514830  0.01    79
 2014-06-21  USC00516128  0.09    73
 ...                 ...   ...   ...
 2014-06-25  USC00519523  0.07    74
 2012-06-09  USC00519281  0.00    74
 2016-06-27  USC00519281  0.37    72
 2010-06-24  USC00514830  0.05    74
 2012-06-27  USC00513117  0.23    71
 
 [1517 rows x 3 columns],
                 station  prcp  tobs
 date                               
 2010-12-01  USC00519397  0.04    76
 2010-12-03  USC00519397  0.00    74
 2010-12-04  USC00519397  0.00    74
 2010-12-06  USC00519397  0.00    64
 2010-12-07  USC00519397  0.00    64
 ...                 ...   ...   ...
 2016-12-27  USC00516128  0.14    71
 2016-12-28  USC00516128  0.14    71
 2016-12-29  USC00516128  1.03    69
 2016-12-30  USC00516128  2.37    65
 2016-12-31  USC00516128  0.90    65
 
 [1517 r

In [75]:
# Run paired t-test
stats.ttest_rel(june_sample_df['tobs'], dec_df['tobs'])

Ttest_relResult(statistic=30.92375981564925, pvalue=3.3070307233144e-163)

In [76]:
#Run independent t-test, using the entirety of June temperatures (not a random sample, because arrays don't have to mach)
stats.ttest_ind(june_df['tobs'], dec_df['tobs'])

Ttest_indResult(statistic=31.60372399000329, pvalue=3.9025129038616655e-191)

### Analysis

The t-value is quite large and the p-value is VERY small when running either a paired t-test (taking a random sample of June temps to make the arrays the same size) OR an independent t-test (uses all data from June). This indicates that the mean temperature in June is significantly different from the mean temperature in December. 